In [75]:
import numpy as np

In [76]:
def conv_single_step(a_slice_prev, W, b):

    s = np.multiply(a_slice_prev,W) 
    # Sum over all entries of the volume s.
    Z = np.sum(s)
    # Add bias b to Z. Cast b to a float() so that Z results in a scalar value.
    Z = Z + float(b)
    return Z

In [77]:
np.random.seed(1)
a_slice_prev = np.random.randn(3, 4, 4)
W = np.random.randn(3, 4, 4)
b = np.random.randn(1, 1, 1)

Z = conv_single_step(a_slice_prev, W, b)
print("Z =", Z)

Z = -6.999089450680221


In [82]:
def convolution_junction(image,kernel,bias,hparameters):
    
    (image_count,number_of_channels,n_H_prev,n_W_prev) = image.shape
    (n_C_prev,n_c,f,f1) = kernel.shape
    
    stride = hparameters['stride']
    pad = hparameters['pad']
    
    X_pad = np.pad(image,((0,0),(0,0),(pad,pad),(pad,pad)),'constant',constant_values = 0)

    output_height = int(np.floor(((n_H_prev+ (2*pad)-f)/stride) + 1))
    output_width = int(np.floor(((n_W_prev+ (2*pad)-f)/stride) + 1))
    
    result = np.zeros(shape=(image_count,n_c,output_height,output_width))
    
    for t in range(0,image_count):
        
        a_prev_image = X_pad[t]
        
        for y in range(0,output_height):
            
                for z in range(0,output_width):
                    
                    for m in range(0,n_c):
                        
                        vert_start = y*stride
                        vert_end = vert_start+f
                        horiz_start = z*stride
                        horiz_end = horiz_start+f
                        
                        
                        a_slice_prev = a_prev_image[:,vert_start:vert_end,horiz_start:horiz_end]
                        
                        result[t, m, y, z] = conv_single_step(a_slice_prev,kernel[:,m,:,:],bias[:,m,:,:])
                        
    cache = (image, kernel, bias, pad, stride)
    
    return result, cache

In [85]:
np.random.seed(1)
image = np.random.rand(1,3,4,4)
#bias & kernel channels should be changed together
kernel = np.random.rand(3,8,2,2)
b = np.random.randn(1,8,1,1)
hparameters = {"pad" : 2,
               "stride": 2}

In [86]:
result,ccc = convolution_junction(image,kernel,b,hparameters)

In [87]:
print("Kernel: ")
print(kernel[0])

#Image changes based on padding
print("Image: ")
print(image[0])

Kernel: 
[[[0.28777534 0.13002857]
  [0.01936696 0.67883553]]

 [[0.21162812 0.26554666]
  [0.49157316 0.05336255]]

 [[0.57411761 0.14672857]
  [0.58930554 0.69975836]]

 [[0.10233443 0.41405599]
  [0.69440016 0.41417927]]

 [[0.04995346 0.53589641]
  [0.66379465 0.51488911]]

 [[0.94459476 0.58655504]
  [0.90340192 0.1374747 ]]

 [[0.13927635 0.80739129]
  [0.39767684 0.1653542 ]]

 [[0.92750858 0.34776586]
  [0.7508121  0.72599799]]]
Image: 
[[[4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01]
  [1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01]
  [3.96767474e-01 5.38816734e-01 4.19194514e-01 6.85219500e-01]
  [2.04452250e-01 8.78117436e-01 2.73875932e-02 6.70467510e-01]]

 [[4.17304802e-01 5.58689828e-01 1.40386939e-01 1.98101489e-01]
  [8.00744569e-01 9.68261576e-01 3.13424178e-01 6.92322616e-01]
  [8.76389152e-01 8.94606664e-01 8.50442114e-02 3.90547832e-02]
  [1.69830420e-01 8.78142503e-01 9.83468338e-02 4.21107625e-01]]

 [[9.57889530e-01 5.33165285e-01 6.

In [88]:
print(result[0])

[[[ 0.74055645  0.74055645  0.74055645  0.74055645]
  [ 0.74055645  4.61661865  3.17556801  0.74055645]
  [ 0.74055645  5.0795541   2.81951707  0.74055645]
  [ 0.74055645  0.74055645  0.74055645  0.74055645]]

 [[-0.9537006  -0.9537006  -0.9537006  -0.9537006 ]
  [-0.9537006   3.09951227  1.17318506 -0.9537006 ]
  [-0.9537006   2.94386908  1.49547885 -0.9537006 ]
  [-0.9537006  -0.9537006  -0.9537006  -0.9537006 ]]

 [[-0.26621851 -0.26621851 -0.26621851 -0.26621851]
  [-0.26621851  2.12336496  1.24663168 -0.26621851]
  [-0.26621851  2.91893481  1.17639201 -0.26621851]
  [-0.26621851 -0.26621851 -0.26621851 -0.26621851]]

 [[ 0.03261455  0.03261455  0.03261455  0.03261455]
  [ 0.03261455  3.77429649  2.30683091  0.03261455]
  [ 0.03261455  3.67896321  2.27274928  0.03261455]
  [ 0.03261455  0.03261455  0.03261455  0.03261455]]

 [[-1.37311732 -1.37311732 -1.37311732 -1.37311732]
  [-1.37311732  1.95521235  0.67115253 -1.37311732]
  [-1.37311732  2.19706515  0.8099074  -1.37311732]
  [-

In [89]:
def pool_forward(A_prev, hparameters, mode = "max"):
    # Retrieve dimensions from the input shape
    (image_count,n_C_prev,n_H_prev, n_W_prev) = A_prev.shape
    
    # Retrieve hyperparameters from "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    # Define the dimensions of the output
    output_height = int(np.floor(((n_H_prev - f)/stride) + 1))
    output_width = int(np.floor(((n_W_prev- f)/stride) + 1))

    # Initialize output matrix A
    pooled = np.zeros(shape=(image_count,n_C_prev,output_height,output_width))          
    
    ### START CODE HERE ###
    # loop over the channels of the output volume
    for t in range(0,image_count):
                
        for y in range(0,output_height):

            for z in range(0,output_width):
                    
                for m in range(0,n_C_prev):
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = y * stride
                    vert_end = vert_start + f
                    horz_start = z * stride
                    horz_end = horz_start + f
                    
                    
                    # Use the corners to define the current slice on the ith training example of A_prev, channel c. (≈1 line)
                    sliced = A_prev[t,m,vert_start:vert_end,horz_start:horz_end]
                    
                    # Compute the pooling operation on the slice. Use an if statment to differentiate the modes. Use np.max/np.mean.
                    if mode == "max":
                        pooled[t,m,y,z] = np.max(sliced)
                    elif mode == 'average':
                        pooled[t,m,y,z] = np.average(sliced)
    ### END CODE HERE ###
   
    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, 2,2)
    
    # Making sure your output shape is correct

    
    return pooled, cache

In [90]:
np.random.seed(1)
A_prev = np.random.randn(2, 3, 4, 4)
hparameters = {"stride" : 2, "f": 3}

A, cache = pool_forward(A_prev, hparameters)
print("mode = max")
print("A =", A)
print()
A, cache = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A =", A)

mode = max
A = [[[[1.74481176]]

  [[1.14472371]]

  [[1.65980218]]]


 [[[0.93110208]]

  [[2.18557541]]

  [[1.19891788]]]]

mode = average
A = [[[[ 0.25831939]]

  [[ 0.00354003]]

  [[-0.3614822 ]]]


 [[[ 0.15471667]]

  [[ 0.24904245]]

  [[ 0.20670492]]]]
